In [26]:
raise NotImplementedError
import os

def delete_pth_files(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.pth'):
                file_path = os.path.join(root, file)
                os.remove(file_path)
                print(f"Deleted: {file_path}")

# 使用示例
delete_pth_files('results/tests_old')


In [15]:
import os
os.system("python show_result.py --result-dir tests_1012 --exp 1_dynst_gt_only")
os.system("python show_result.py --result-dir tests_1012 --exp 2_dynst_gt_only")
os.system("python show_result.py --result-dir tests_1012 --exp 3_dynst_gt_only")

从 results/tests_1012/exp_1_dynst_gt_only 中提取结果

[err_test] England,France,Italy,Spain

7->1 (w7s2) | 7->1 (w7s6) | 7->1 (w7s13)
[    dynst] 2.292	0.98	-	-	2.827	1.314	5.59	-	7.964	1.746	5.475	11.957

从 results/tests_1012/exp_2_dynst_gt_only 中提取结果

[err_test] England,France,Italy,Spain

7->1 (w7s2) | 7->1 (w7s6) | 7->1 (w7s13)
[    dynst] 0.494	0.346	0.163	1.538	0.715	0.555	0.334	1.155	0.841	1.669	0.507	1.709

从 results/tests_1012/exp_3_dynst_gt_only 中提取结果

[err_test] England,France,Italy,Spain

7->1 (w7s2) | 7->1 (w7s6) | 7->1 (w7s13)
[    dynst] 1.171	0.24	0.23	1.86	0.861	0.334	0.297	0.974	0.395	0.59	0.42	1.724



0